In [1]:
import math
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

/home/indrajit/.virtualenvs/fb/local/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
df_train = pd.read_csv('../Kaggle_Datasets/Facebook/train.csv')
df_test = pd.read_csv('../Kaggle_Datasets/Facebook/test.csv')

In [3]:
mini_train = df_train.sample(100000, random_state=77)
train, test = train_test_split(mini_train, random_state=88)

In [4]:
mean_train = train.groupby(['place_id']).mean()

In [5]:
def find_mean_x(place_id):
    return mean_train.loc[place_id]['x']

def find_mean_y(place_id):
    return mean_train.loc[place_id]['y']
    
train.loc[:, 'mean_x'] = train['place_id'].apply(find_mean_x)
train.loc[:, 'mean_y'] = train['place_id'].apply(find_mean_y)
train.head()

/home/indrajit/.virtualenvs/fb/local/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/indrajit/.virtualenvs/fb/local/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,row_id,x,y,accuracy,time,place_id,mean_x,mean_y
11595784,11595784,0.6684,9.3710,65,206270,2566588249,0.645250,9.362025
17116832,17116832,4.2659,9.9657,29,658949,3228485175,4.242000,9.968100
14228893,14228893,3.8986,9.9860,72,737430,8493860112,3.867300,9.983400
12584487,12584487,2.6214,0.7714,69,309534,2882690398,2.624250,0.770150
4488058,4488058,3.4351,0.8225,50,84545,6958273791,3.764957,0.849757


In [6]:
std_train = train.groupby(['place_id']).std().fillna(0)

In [7]:
def find_std_x(place_id):
    return std_train.loc[place_id]['x']

def find_std_y(place_id):
    return std_train.loc[place_id]['y']
    
train.loc[:, 'std_x'] = train['place_id'].apply(find_std_x)
train.loc[:, 'std_y'] = train['place_id'].apply(find_std_y)
train.head()

,row_id,x,y,accuracy,time,place_id,mean_x,mean_y,std_x,std_y
11595784,11595784,0.6684,9.3710,65,206270,2566588249,0.645250,9.362025,0.037236,0.007781
17116832,17116832,4.2659,9.9657,29,658949,3228485175,4.242000,9.968100,0.033800,0.003394
14228893,14228893,3.8986,9.9860,72,737430,8493860112,3.867300,9.983400,0.044265,0.003677
12584487,12584487,2.6214,0.7714,69,309534,2882690398,2.624250,0.770150,0.004031,0.001768
4488058,4488058,3.4351,0.8225,50,84545,6958273791,3.764957,0.849757,0.329522,0.013683


In [8]:
train.loc[:, 'hour'] = train.time // 60 % 24 + 1
train.loc[:, 'day'] = train.time // (60*24) % 7 + 1
train.loc[:, 'week'] = train.time // (60*24*7) % 52 + 1
train.loc[:, 'month'] = train.time // (60*24*30) % 12 + 1
train.loc[:, 'year'] = train.time // (60*24*365) + 1

train.loc[:, 'hours'] = train.time // 60 + 1
train.loc[:, 'days'] = train.time // (60*24) + 1
train.loc[:, 'weeks'] = train.time // (60*24*7) + 1
train.loc[:, 'months'] = train.time // (60*24*30) + 1

train.head()

,row_id,x,y,accuracy,time,place_id,mean_x,mean_y,std_x,std_y,hour,day,week,month,year,hours,days,weeks,months
11595784,11595784,0.6684,9.3710,65,206270,2566588249,0.645250,9.362025,0.037236,0.007781,6,4,21,5,1,3438,144,21,5
17116832,17116832,4.2659,9.9657,29,658949,3228485175,4.242000,9.968100,0.033800,0.003394,15,3,14,4,2,10983,458,66,16
14228893,14228893,3.8986,9.9860,72,737430,8493860112,3.867300,9.983400,0.044265,0.003677,3,2,22,6,2,12291,513,74,18
12584487,12584487,2.6214,0.7714,69,309534,2882690398,2.624250,0.770150,0.004031,0.001768,23,5,31,8,1,5159,215,31,8
4488058,4488058,3.4351,0.8225,50,84545,6958273791,3.764957,0.849757,0.329522,0.013683,18,3,9,2,1,1410,59,9,2


In [9]:
train.loc[:, 'log2_accuracy'] = np.log2(train.accuracy) * 10
train.loc[:, 'log10_accuracy'] = np.log10(train.accuracy) * 10
train.head()

,row_id,x,y,accuracy,time,place_id,mean_x,mean_y,std_x,std_y,...,day,week,month,year,hours,days,weeks,months,log2_accuracy,log10_accuracy
11595784,11595784,0.6684,9.3710,65,206270,2566588249,0.645250,9.362025,0.037236,0.007781,...,4,21,5,1,3438,144,21,5,60.223678,18.129134
17116832,17116832,4.2659,9.9657,29,658949,3228485175,4.242000,9.968100,0.033800,0.003394,...,3,14,4,2,10983,458,66,16,48.579810,14.623980
14228893,14228893,3.8986,9.9860,72,737430,8493860112,3.867300,9.983400,0.044265,0.003677,...,2,22,6,2,12291,513,74,18,61.699250,18.573325
12584487,12584487,2.6214,0.7714,69,309534,2882690398,2.624250,0.770150,0.004031,0.001768,...,5,31,8,1,5159,215,31,8,61.085245,18.388491
4488058,4488058,3.4351,0.8225,50,84545,6958273791,3.764957,0.849757,0.329522,0.013683,...,3,9,2,1,1410,59,9,2,56.438562,16.989700


In [10]:
def apply_accu_class(x):
    rise_ranges = ((50, 90), (130, 160), (220, 310), (380, 600))
    fall_ranges = ((0, 40), (110, 120), (190, 210), (320, 370), )
    peak_ranges = ((90, 110), (160, 190), (310, 320), )
    trough_ranges = ((40, 50), (120, 130), (210, 220), (370, 380))
    class_dict = {rise_ranges: 1, fall_ranges: -1, peak_ranges: 2, trough_ranges: -2}
    for rang in [rise_ranges, fall_ranges, peak_ranges, trough_ranges]:
        for a, b in rang:
            if a <= x < b:
                return class_dict[rang]

In [11]:
train.loc[:, 'accu_class'] = train.days.apply(apply_accu_class)

In [12]:
train.head()

,row_id,x,y,accuracy,time,place_id,mean_x,mean_y,std_x,std_y,...,week,month,year,hours,days,weeks,months,log2_accuracy,log10_accuracy,accu_class
11595784,11595784,0.6684,9.3710,65,206270,2566588249,0.645250,9.362025,0.037236,0.007781,...,21,5,1,3438,144,21,5,60.223678,18.129134,1
17116832,17116832,4.2659,9.9657,29,658949,3228485175,4.242000,9.968100,0.033800,0.003394,...,14,4,2,10983,458,66,16,48.579810,14.623980,1
14228893,14228893,3.8986,9.9860,72,737430,8493860112,3.867300,9.983400,0.044265,0.003677,...,22,6,2,12291,513,74,18,61.699250,18.573325,1
12584487,12584487,2.6214,0.7714,69,309534,2882690398,2.624250,0.770150,0.004031,0.001768,...,31,8,1,5159,215,31,8,61.085245,18.388491,-2
4488058,4488058,3.4351,0.8225,50,84545,6958273791,3.764957,0.849757,0.329522,0.013683,...,9,2,1,1410,59,9,2,56.438562,16.989700,1


In [13]:
train.columns

Index([u'row_id', u'x', u'y', u'accuracy', u'time', u'place_id', u'mean_x',
       u'mean_y', u'std_x', u'std_y', u'hour', u'day', u'week', u'month',
       u'year', u'hours', u'days', u'weeks', u'months', u'log2_accuracy',
       u'log10_accuracy', u'accu_class'],
      dtype='object')

In [14]:
attributes = [u'x', u'y', u'accuracy', u'time', u'place_id', u'mean_x',
       u'mean_y', u'std_x', u'std_y', u'hour', u'day', u'week', u'month',
       u'year', u'log2_accuracy', u'log10_accuracy', u'accu_class']

In [15]:
model = RandomForestClassifier(n_estimators=5, warm_start=True, random_state=3, n_jobs=1)

In [16]:
train.loc[:, attributes].head()

,x,y,accuracy,time,place_id,mean_x,mean_y,std_x,std_y,hour,day,week,month,year,log2_accuracy,log10_accuracy,accu_class
11595784,0.6684,9.3710,65,206270,2566588249,0.645250,9.362025,0.037236,0.007781,6,4,21,5,1,60.223678,18.129134,1
17116832,4.2659,9.9657,29,658949,3228485175,4.242000,9.968100,0.033800,0.003394,15,3,14,4,2,48.579810,14.623980,1
14228893,3.8986,9.9860,72,737430,8493860112,3.867300,9.983400,0.044265,0.003677,3,2,22,6,2,61.699250,18.573325,1
12584487,2.6214,0.7714,69,309534,2882690398,2.624250,0.770150,0.004031,0.001768,23,5,31,8,1,61.085245,18.388491,-2
4488058,3.4351,0.8225,50,84545,6958273791,3.764957,0.849757,0.329522,0.013683,18,3,9,2,1,56.438562,16.989700,1


In [ ]:
model.fit(train.loc[:, attributes], train.place_id)